In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init() 
path = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/'
deepct = pyterrier_deepct.DeepCTTransformer(path + "bert_files/bert-base-uncased/bert_config.json", "/workspace/src/evaluate_deepct/marco/model.ckpt-65816", vocab_file = path + "bert_files/bert-base-uncased/vocab.txt", max_seq_length=512)
import pandas as pd
path = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/'
splitted_df = pd.read_csv(path + 'splitted_df.csv')
splitted_df = splitted_df.iloc[: , 1:]
splitted_df['text'] = splitted_df['text'].astype(str)
splitted_df['docno'] = splitted_df['docno'].astype(str)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2srs2kf0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba1edf0470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, 

In [9]:
splits = np.array_split(splitted_df, 3)
splits_transformed = []
for split in splits:
    splits_transformed.append(deepct.transform(split))

INFO:tensorflow:Writing example 0
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: Sb38112c8-A443a9828
INFO:tensorflow:tokens: [CLS] done . [SEP]
INFO:tensorflow:input_ids: 101 2589 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [13]:
concat = pd.concat(splits_transformed)
transformed = concat.copy()
transformed['text'] = concat.groupby(['docno'])['text'].transform(lambda x : ' '.join(x))
transformed = transformed.drop_duplicates()
transformed

,docno,text
0,Sb38112c8-A443a9828,done done done done done done done done done d...
1,Sb38112c8-A48f87c39,let let let ' ' ' ' ' s s s s s s s get get ge...
2,Sb38112c8-A85a910e8,not stated stated stated stated stated stated ...
3,Sb38112c8-A96d2d2c7,schools schools schools schools schools school...
4,S4468ab0b-Af7993e79,impasse impasse impasse rounds rounds rounds 1...
...,...,...
831753,S18f76536-A8ed8e1,marks anniversary anniversary important ruling...
831754,S1aa1675d-A8c1dd45a,anniversary anniversary police police police p...
831755,S151e639d-A433b698c,1948 1948 world world world world world world ...
831756,S18eb2e1a-Afe2ebde7,canada canada canada canada canada canada cana...


In [14]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles = []
title_ids = []
nums = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
topics = pd.DataFrame()
topics['qid'] = nums
topics['query'] = titles

qrels_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

docnos = qrels.docno

In [15]:
save_path = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/'
# df_test = pd.read_csv(save_path + 'topics_conclusions_transformed.csv')

path_targer = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/'
pd_indexer = pt.DFIndexer(path_targer + "pd_index_deepct-marco")
indexref = pd_indexer.index(transformed["text"], transformed["docno"])
# indexref = pt.IndexRef.of("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_deepct-conclusions1/data.properties")

index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)

res

20:38:29.891 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (Sb38112c8-A443a9828) - further warnings are suppressed
20:42:36.924 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1536 empty documents


,qid,docid,docno,rank,score,query
0,1,316537,Sff0947ec-A46d54897,0,19.285887,should teachers get tenure
1,1,129070,S51530f3f-Ae32a4a1b,1,19.275619,should teachers get tenure
2,1,307910,Sc065954f-A39b0539e,2,19.272086,should teachers get tenure
3,1,129066,S51530f3f-A4715d76f,3,19.271001,should teachers get tenure
4,1,88183,Sb0680508-Aa5189771,4,19.270403,should teachers get tenure
...,...,...,...,...,...,...
98995,100,295970,S8a4c9f41-Af0619166,995,4.551569,do we need cash
98996,100,33577,S2aadf1e8-A6a500bb8,996,4.550496,do we need cash
98997,100,208109,Sb8d9e822-A3d3ad205,997,4.548265,do we need cash
98998,100,49118,Se0ccd39d-Ab530103,998,4.540873,do we need cash


In [16]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])
eval

{'ndcg_cut_5': 0.4613670486988758,
 'bpref': 0.6868795531672882,
 'ndcg_cut_25': 0.41737703253888275}

In [17]:
res_copy = res.copy()
res_copy = res_copy[res_copy['docno'].isin(docnos)]
res_copy.head(10)

eval = pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])
eval

{'ndcg_cut_5': 0.5731244186529317,
 'bpref': 0.6868795531672882,
 'ndcg_cut_25': 0.6419491320981001}